In [2]:
# Import Libraries
import pandas as pd
import numpy as np

import requests
import json
import datetime
import itertools
from itertools import repeat
import plotnine
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,6

In [107]:
#read in all data:
dataE = pd.read_csv("dataE.csv")
dataI = pd.read_csv("dataI.csv")
dataZ = pd.read_csv("dataZ.csv")
dataM = pd.read_csv("dataM.csv")
dataR = pd.read_csv("dataR.csv")
dataS = pd.read_csv("dataS.csv")

In [109]:
#set date as index:
dataE2 = dataE.copy()
dataE = dataE.set_index("Date")
dataE2= dataE2.groupby("Date").mean()

dataI2 = dataI.copy()
dataI = dataI.set_index("Date")
dataI2 = dataI2.groupby("Date").mean()

dataZ2 = dataZ.copy()
dataZ = dataZ.set_index("Date")
dataZ2 = dataZ2.groupby("Date").mean()

dataM2 = dataM.copy()
dataM = dataM.set_index("Date")
dataM2 = dataM2.set_index("Date").mean()

dataR2 = dataR.copy()
dataR = dataR.set_index("Date")
dataR2= dataR2.groupby("Date").mean()

dataS2 = dataS.copy()
dataS = dataS.set_index("Date")
dataS2= dataS2.groupby("Date").mean()

In [112]:
#delete duplicate rows from each dataset
dataE = dataE.reset_index()
dataE = dataE.drop_duplicates(subset=['Date'],keep='first')
dataE = dataE.set_index("Date")

dataI = dataI.reset_index()
dataI = dataI.drop_duplicates(subset=['Date'],keep='first')
dataI = dataI.set_index("Date")

dataZ = dataZ.reset_index()
dataZ = dataZ.drop_duplicates(subset=['Date'],keep='first')
dataZ = dataZ.set_index("Date")

dataM = dataM.reset_index()
dataM = dataM.drop_duplicates(subset=['Date'],keep='first')
dataM = dataM.set_index("Date")

dataR = dataR.reset_index()
dataR = dataR.drop_duplicates(subset=['Date'],keep='first')
dataR = dataR.set_index("Date")

dataS = dataS.reset_index()
dataS = dataS.drop_duplicates(subset=['Date'],keep='first')
dataS = dataS.set_index("Date")

In [114]:
#perform join on both datasets:
dataE = dataE.join(dataE2, lsuffix="_left", rsuffix="_right")
dataI = dataI.join(dataI2, lsuffix="_left", rsuffix="_right")
dataZ = dataZ.join(dataZ2, lsuffix="_left", rsuffix="_right")
#dataM = dataM.join(dataM2, lsuffix="_left", rsuffix="_right")
dataR = dataR.join(dataR2, lsuffix="_left", rsuffix="_right")
dataS = dataS.join(dataS2, lsuffix="_left", rsuffix="_right")

In [115]:
#extract columns of interest:
dataE = dataE[["tmax_left","tmin_left","Precip_right","average_left"]]
dataI = dataI[["tmax_left","tmin_left","Precip_right","average_left"]]
dataZ = dataZ[["tmax_left","tmin_left","Precip_right","average_left"]]
dataR = dataR[["tmax_left","tmin_left","Precip_right","average_left"]]
dataS = dataS[["tmax_left","tmin_left","Precip_right","average_left"]]
dataM = dataM.rename(columns={
    "tmax":"tmax_left",
    "tmin":"tmin_left",
    "Precip":"Precip_right",
    "average":"average_left"
})


In [116]:
#add in county data:
dataE["county"] = "El Dorado"
dataI["county"] = "Imperial"
dataZ["county"] = "Merced"
dataM["county"] = "Mendocino"
dataR["county"] = "Riverside"
dataS["county"] = "Siskyou"


In [117]:
print(len(dataE))
print(len(dataI))
print(len(dataZ))
print(len(dataM))
print(len(dataR))
print(len(dataS))

299
471
500
294
35
159


In [118]:
#concatenate into one dataset:
weather = pd.concat([dataE,dataI,dataZ,dataM,dataR,dataS])

In [137]:
weather = weather.reset_index()

In [138]:
#load in wind data:
windE = pd.read_csv("el_dorado_county_wind_data.csv")
windI = pd.read_csv("Imperial_county_wind_data.csv")
windZ = pd.read_csv("Merced_county_wind_data.csv")
windM = pd.read_csv("Mendocino_county_wind_data.csv")
windR = pd.read_csv("Riverside_county_wind_data.csv")
windS = pd.read_csv("Siskyou_county_wind_data.csv")


In [139]:
#add county names to wind data:
windE["county"] = "El Dorado"
windI["county"] = "Imperial"
windZ["county"] = "Merced"
windM["county"] = "Mendocino"
windR["county"] = "Riverside"
windS["county"] = "Siskyou"

In [140]:
#merge all wind data together:
winds = pd.concat([windE,windI,windZ,windM,windR,windS])

In [141]:
#set indices for join:
winds = winds.set_index(["Date","county"])
weather = weather.set_index(["Date","county"])

In [143]:
weather

,,tmax_left,tmin_left,Precip_right,average_left
Date,county,,,,
2003-01-01,El Dorado,50.0,42.0,0.080000,46.0
2003-01-02,El Dorado,44.0,29.0,0.042727,36.5
2003-01-03,El Dorado,53.0,31.0,0.000000,42.0
2003-01-04,El Dorado,64.0,42.0,0.000000,53.0
2003-01-05,El Dorado,69.0,44.0,0.009091,56.5
...,...,...,...,...,...
2010-02-05,Siskyou,50.0,30.0,0.416875,40.0
2010-02-06,Siskyou,51.0,38.0,0.115000,44.5
2010-02-07,Siskyou,50.0,35.0,0.013571,42.5


In [144]:
#join weather and wind data
joined_weather = weather.join(winds)

In [146]:
#reset index
joined_weather = joined_weather.reset_index()

In [152]:
#save to csv:
joined_weather.to_csv("joined_weather.csv")